# **Introduction**
#### [For a detailed presentation of the analysis and results see the published PDF paper.]


The **goal** of this project is to build a deep learning network classifier which is able to classify order and chaos in 2-dimensional, discreet orbits on the plane, which result from Hamiltonian systems. Such orbits result from the so--called Poicare sections or Poincare maps.

As our reference map we use the so--called **Standard Map** which is one of the most representative Poincare maps, and exhibits universal features of the way order and chaos emerges in Hamiltonian systems. We first use it to train and validate our deep model, and then we apply our trained model on other Poincare maps. We show that the trained ML model is able to identify chaos and order in other systems too with reasonably high accuracy, due to the **universality features** of chaotic/ordered dynamics of Hamiltonian systems. 

#### **The code's modules:**

 - functions.py: Definitions of functions generating the Poincare maps 

 - generate_data.py: Data preparation and augmentation finctionality

 - model.py: Deep network model, and testing functionality 

 - Main_notebook.ipynb: Main analysis + plots 


### ****Step I**:** Definition of data-generating functions (functions.py)

Here we load the module with the **Standard Map** and the other Poincare maps we use. Given the initial parameters/conditions their output will be a sequence of points corresponding to the orbit system on the 2D plane. When loading the data, the arrays with the orbits  are generated on the spot - they are not pre-saved in tabulated form.

In all maps, **K** is the constant parametrising the departure from integrability in the given system, i.e the perturbation away from the integrable system which leads to chaos. **$n\_points$** is the number of iterations of the map. **$(x0,y0)$** is the initial conditions on the 2-dimensional Poincare surface. Different choices of initial parameters result to dynamics which exhibit order and chaos respectively. We also give the option of adding a small **Gaussian noise** to the training data (Standard Map) so that the model generalises better. The data exported for training are in the form **(res)x(res)**, where "res" is an integer defining the resolution ("pixels") along the x- and y- direction on the 2D plane of the orbit. We choose 30x30 (see also paper for more information).

In [1]:
import numpy as np
from generate_data import *
from functions import *
from model import * 
import matplotlib.pyplot as plt

n_points = 450 #450 # number of evaluations points for x and y. Total points of map = 2*n_points.
res = int(np.sqrt(2*n_points)) # must be integer. This will be the resolution of the 'image'

2024-01-28 17:30:19.531481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 17:30:19.531525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 17:30:19.557853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 17:30:19.620113: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# **Step II: Preparing the training data** (generate_data.py)

Here we set up the arrays with the training and validation data. We choose the **Standard Map** as our generator for the training data (see paper). The set cosists of **representative orbits of order and chaos**, specifying appropriate values for the order paramerter $K$ and initial points $(x0,y0)$. For the **labels** of the training set we choose: (order, chaos) = (1, 0). The training data are fed into the ML network through an **ImageGenerator** which augments the data (rotation, shuffling, etc) to achieve better generalisation. 

**Training** = (2/3)*(total data),      **Validation** = (1/3)*(total data).

Later, we loop over different agumentation of the training/validation set at fixed agumentation parameters, in order to test that the resulting sets lie around a similar standard deviation $\sigma$. 

In [2]:
from sklearn.utils import shuffle

data_SM = load_data_SM() # load Standard Map data
data_deVog = load_data_deVog() # load de Vogelaere Map data
Np = int(len(data_SM[0])) 
frac = 2/3 # fraction of dataset to use for training, the rest (1-frac) is for validation

X_train = data_SM[0] # train/validation orbits
y = data_SM[1] # train/validation labels
data_SM = shuffle(X_train, y) # shuffle training/validation data
data_train = [d[:int((frac)*Np)] for d in data_SM] # training data
data_val = [d[-int((1-frac)*Np):] for d in data_SM] # validation data
data_test = data_deVog # benchmark testing data
data_test_Web = load_data_Web() # more testing data 


In [3]:
# Iterator for training with augmentation.
iterator_train =  produce_iterator(data_train, batch_size=64, print_stats=True,
                  shuffle=True, rotation_range=60, 
                  width_shift_range=0.001, height_shift_range=0.001, 
                  shear_range=0.35, fill_mode='nearest')

# We choose the same data generator for training and validation set.
#iterator_val =  produce_iterator(data_val, batch_size=64, shuffle = True, print_stats=True)
iterator_val =  produce_iterator(data_val, batch_size=64, print_stats=True,
                  shuffle=True, rotation_range=60, 
                  width_shift_range=0.001, height_shift_range=0.001, 
                  shear_range=0.35,fill_mode='nearest')

# Test iterators. No augmentation used.
iterator_test = produce_iterator(data_test, batch_size=384, shuffle = True, print_stats=True)

iterator_test_Web = produce_iterator(data_test_Web, batch_size=384, shuffle = True, print_stats=True)


Dataset statistics:  (64, 30, 30, 1) -0.026402855 0.8721529
Dataset statistics:  (64, 30, 30, 1) -0.008044084 0.86294127
Dataset statistics:  (384, 30, 30, 1) -0.0024601517 0.99213046
Dataset statistics:  (384, 30, 30, 1) 1.987118e-05 1.0014379


In [ ]:
# Here we want to produce different realisations of the training data using augmentation/standardisation
# which produces a standardised version of the data with a mean and std.
# We check how the mean and std of the data set varies. We would like that to see reasonable scatter.

stats = []
for i in range(0,50):
    iterator =  produce_iterator(data_train, batch_size=64, print_stats=True,
                shuffle=True, rotation_range=90, 
                width_shift_range=0.001, height_shift_range=0.001, 
                shear_range=0.3, fill_mode='nearest')

    mean_i, std_i = train_data_stats(iterator, 'train')
    stats.append([mean_i, std_i])

stats = np.array(stats)

# Plot the mean and std.
plt.scatter(stats[:,0],stats[:,1])
plt.xlabel('μ_augm.', fontsize=20, color='black') 
plt.ylabel('σ_augm.', fontsize=20, color='black')

print('<μ_augm> = ', np.mean(stats[:,0])) # The mean value of the standardised mean of each dataset realisation 
print('σ of <μ_augm> = ', np.std(stats[:,0]))  # The std value of the standardised std of each dataset realisation 
print('<σ_augm> = ', np.mean(stats[:,1])) # The mean value of the standardised mean of each dataset realisation 
print('σ of  <σ_augm> = ', np.std(stats[:,1]))  # The std value of the standardised std of each dataset realisation 

# **Step III: Building of the deep network** (model.py)

Here we construct the **order-chaos classifier ML network**. We use a simple feed-forward architecture combining convolutional layers with a set of dense layers. (See paper for more.)

In [ ]:
### Create the deep model
model, history = produce_model(iterator_train, iterator_val, epochs = 70)

# Plot metrics
plot_history(history)

# **Step IV: Testing** (model.py)
 
**i)** We first test the model with our **benchmark data set**, a dataset based on the the so--called  **quadratic de Vogelaere map**, which correspond to another 2-dimensional Poincare map (see paper). We follow a similar approach as before. We also test against another independent map, the **Web map**.

**ii)** We **loop** over train-validation-evaluation for our **benchmark data set** in order to produce a statistical sample of evaluation results based on different trained models. This will provide a reassurance given the inherit stochasticity in the models.  

**iii)** We then test against **JHMAP map** (see paper).

In [ ]:
###### Save the trained model if needed ######
#model.save('model_22-12-2023_acc95.keras')

###### Load the saved model if needed ######
new_model = tf.keras.models.load_model('best_model_acc95.0_epoch45.keras')
model = new_model

# Evaluate the model with the benchmark testing set (deVogelaere map)
test_1 = produce_evaluation(iterator_test, model)
print("Benchmark test = ", test_1)

# Evaluate the model with the Web map 
test_2 = produce_evaluation(iterator_test_Web, model)
print("Web map test = ", test_2)


In [ ]:
### Create a statistical sample of train+evaluation. 
# It loops over training and then evaluating the resulting model. 

import warnings
warnings.filterwarnings('ignore')

# similar as the iterator_test defined previously, but with larger batch size 
iterator_test = produce_iterator(data_test, batch_size = 256, shuffle = True, print_stats=True)
acc_results = []
acc_results = [[i,loop_train_eval(i, data_train, data_val, iterator_train, iterator_val, iterator_test, epochs = 45)] for i in range(0,40)]

# Plot the results
array1=np.array(acc_results)
plt.scatter(array1[:,0],array1[:,1])
print('Accuracy mean = ', np.mean(array1[:,1]))
print('Accuracy std = ', np.std(array1[:,1]))


#### Here we test against the JHMAP. 

In [ ]:
################ Import the files and prepare the list with data 
data = (('001', 1), ('002',1), ('003',0), ('004',0),('005',0),('006',0),('007',1), ('008',1),('009',1),('010',1),('011',0),('012',0),('013',0),('014',0),('015',0),('016',0),('017',0),('018',0),('019',1),('020',1),('021',1) )
data_JHMAP_orbits = [] 
data_JHMAP_labels = [] 
# This loop imports the data, normalises them and appends them in a list
for data_i in data: 
    file_i = data_i[0]
    class_i = data_i[1]
    folder_i = '/home/saltas/chaos/final/external_orbits/JHMAP' + file_i + '.dat' 
    test_i = np.loadtxt(folder_i)[:n_points]
    testX = [] 
    for i in range(0,len(test_i)):
        testX.append([np.float64(test_i[i][0])/max(np.abs(test_i[:,0])),np.float64(test_i[i][1])/max(np.abs(test_i[:,1]))])
    testX = np.array(testX)
    testX = testX.reshape(res,res)
    data_JHMAP_orbits.append(testX)
    data_JHMAP_labels.append(class_i)
data_JHMAP_orbits = np.array(data_JHMAP_orbits)
data_JHMAP_labels = np.array(data_JHMAP_labels)
data_JHMAP =  data_JHMAP_orbits, data_JHMAP_labels
iterator_test_JHMAP = produce_iterator(data_JHMAP, batch_size=9)

produce_evaluation(iterator_test_JHMAP, model)


## Initial conditions to reproduce plots in the paper
**Maps**: Standard map, de Vogelaerre map, Web map.


In [ ]:
K0 = 0.1   # The perturbation parameter in each map
x00 = 0.01 # Initial condition on x-axis
y00 = 0.0  # Initial condition on y-axis

# Web map - chaos : [K 0 = 3.5, 5.45, 8.5, 9.6, x0,y0 = default ] #n_points = 100
##
# deVog map - chaos : [K0 = -1.042, x0 = 0.01, y0 =0.] , [K0 = -1.04, x0 = 0.1, y0 =0.05] 
# deVog map - order : [K0 = 0.504, x0 = 0.2, y0 =0.2] ,  [K0 = -1.0515, x0 = -0.1, y0 =0.005]
##
# Standard Map - chaos : [K0 = 0.93, x0 = np.pi, y0 = 8.]  #n_points = 100, [K0 = 0.95, x0 = np.pi, y0 = -4.1]  #n_points = 50                   
# Standard Map - order :  [K0 = 0.8, np.pi, 8.35], [K0 = 1.2, np.pi, 0.5]
#
#test0_plot = Standard_map(K0, n_points =  150*n_points, plot = True, x0 = x00, y0 = y00 )
#test0_plot = deVog_map(K0, n_points = int(n_points), plot = True, x0 = x00, y0 = y00 )
#test0_plot = Web_map(K0, q = 4, n_points =  100*n_points, plot = True )

plt.scatter(test0_plot[0],test0_plot[1], s = 0.1, color = 'royalblue')